In [8]:
%cd /panfs/ccds02/nobackup/people/sbhusha1/ostfl/maxar/
asp_exec = "/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/"

/panfs/ccds02/nobackup/people/sbhusha1/ostfl/maxar


In [2]:
import os,glob,sys
import subprocess

In [3]:
from dgtools import dglib

In [4]:
stereo_dir = 'WV02WV02_20091222_10300100023BC100_1030010003D22F00/'
pair_metadata = dglib.dir_pair_dict(stereo_dir)

In [11]:
id1 = pair_metadata['id1_dict']['id']
gsd1 = pair_metadata['id1_dict']['gsd']
id2 = pair_metadata['id2_dict']['id']
gsd2 = pair_metadata['id2_dict']['gsd']

if gsd1<gsd2:
    #ID1 image is finer resolution
    left_img = sorted(glob.glob(os.path.join(stereo_dir,f"{id1}*r100.tif")))[0]
    right_img = sorted(glob.glob(os.path.join(stereo_dir,f"{id2}*r100.tif")))[0]
else:
    #ID2 image is finer resolution
    left_img = sorted(glob.glob(os.path.join(stereo_dir,f"{id2}*r100.tif")))[0]
    right_img = sorted(glob.glob(os.path.join(stereo_dir,f"{id1}*r100.tif")))[0]

left_xml = os.path.splitext(left_img)[0]+".xml"
right_xml = os.path.splitext(right_img)[0]+".xml"

## Question 1: How does OpenCV Block Matching & SGM compare to ASP Block Matching & SGM?

* First experiment is over a flat site with narrow convergence angle pair, where "ASP-special pre-stereo orthorectification" is not required
* OpenCV Block matching results already exist, here we prepare jobs for
    * OpenCV SGM
    * ASP Block matching
    * ASP SGM

For reference, this is the opencv_bm which started before this notebook was made
#! /bin/bash
#SBATCH --job-name=opencv_bm -t 15:00:00 -c10 --mem=190G

cd /panfs/ccds02/nobackup/people/sbhusha1/ostfl/maxar/WV02WV02_20091222_10300100023BC100_1030010003D22F00

/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo -t rpc --stereo-algorithm "opencv_bm -block_size 21 -texture_thresh 10 -prefilter_cap 31 -uniqueness_ratio 15 -speckle_size 100 -speckle_range 32 -disp12_diff 1" --alignment-method local_epipolar --corr-seed-mode 1 --sgm-collar-size 0 --corr-tile-size 1024 1030010003D22F00.r100.tif 1030010003D22F00.r100.xml 10300100023BC100.r100.tif 10300100023BC100.r100.xml opencv_bm/opencv_bm_processing

## OpenCV SGM

In [13]:
#! /bin/bash #SBATCH --job-name=opencv_bm -t 15:00:00 -c10 --mem=190G


parent_dir = "/panfs/ccds02/nobackup/people/sbhusha1/ostfl/maxar/"

In [15]:

job_fn = f"opencv_sgbm_job.sh"
output_directory = os.path.join(stereo_dir,'cpu/opencv_sgbm/opencv_sgbm_processing')


opencv_alg1 = "opencv_sgbm -mode sgbm -block_size 7 -P1 8 -P2 32"
opencv_alg2 = " -prefilter_cap 63 -uniqueness_ratio 10 -speckle_size 100"
opencv_alg3 = " -speckle_range 32 -disp12_diff 1"
opencv_alg = opencv_alg1 + opencv_alg2 + opencv_alg3


opencv_sgm_opt = f' -t rpc --alignment-method local_epipolar --stereo-algorithm "{opencv_alg}"'
opencv_args = f' {left_img} {left_xml} {right_img} {right_xml} {output_directory}'

parallel_stereo_call = "/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo"
parallel_stereo_cmd = f'time {parallel_stereo_call} {opencv_sgm_opt} {opencv_args}'

output_pointcloud = f"{output_directory}*-PC.tif"

p2dem_call = "/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/point2dem"
p2dem_opts = "--tr 4.0 --nodata-value -9999.0 --t_srs EPSG:32616 --errorimage"
p2dem_cmd = f'time {p2dem_call} {p2dem_opts} {output_pointcloud}'

with open(job_fn,'w') as f:
    slurm_keyword = f'#! /bin/bash \n #SBATCH --job-name={os.path.basename(job_fn)} -t 20:00:00 -c10 --mem=190G'
    directory_change = f'cd {parent_dir}'
    out_str = slurm_keyword+"\n"+directory_change+"\n"+parallel_stereo_cmd+"\n"+p2dem_cmd
    f.write(out_str)

## ASP SGM
* Note that we can not try to have each and every parameter for processing the same between different software algorithms
* We will rely on keeping the correlation kernel size consistent for a given processing algorithm.

* In our SGM runs, both ASP and OpenCV will use 7 by 7 px kernels
* In our block matching runs, both ASP and OpenCV will use 21 by 21 px kernels

/vast_swbuild/swbuild3/sbhusha1/pip_git_sw/StereoPipeline-3.5.0-alpha-2024-11-25-x86_64-Linux/libexec/stereo_corr --corr-memory-limit-mb 3000 --cost-mode 3 --corr-tile-size 1024 --stereo-algorithm asp_mgm --num-matches-from-disparity 400000 -t csmmapcsm --subpixel-mode 9 --corr-kernel 7 7 --subpixel-kernel 15 15 --corr-max-levels 5 --corr-timeout 480 --filter-mode 1 --erode-max-size 0 --alignment-method None 104001006E942500_ortho_0.32m.tif 104001006D788E00_ortho_0.32m.tif csm_aligned2refdem-csm-104001006E942500.r100.adjusted_state.json csm_aligned2refdem-csm-104001006D788E00.r100.adjusted_state.json dem_ortho_0.32m/20210920_1819_104001006E942500_104001006D788E00 /nobackupp28/sbhusha1/stv/pcd/coincident_usgs/WV03WV03_20210920_104001006E942500_104001006D788E00/refdem/COP30_E.tif --corr-seed-mode 1 --sgm-collar-size 256

In [20]:
job_fn = f"asp_sgm_job.sh"
output_directory = os.path.join(stereo_dir,'cpu/asp_sgm/asp_sgm_processing')
corr_kernel = 7


## ASP algorithm specific parameters
stereo_algorithm = "asp_sgm"
subpix_kernel = 15
subpix_mode = 9
filter_mode = 1 
erode_max_size = 0
cost_mode = 3
corr_mem_lim = 3000

asp_opt1 = f"--corr-kernel {corr_kernel} {corr_kernel} --subpixel-mode {subpix_mode}"
asp_opt2 = f" --subpixel-kernel 15 15 --filter-mode {filter_mode} --cost-mode {cost_mode}"
asp_opt3= f" --corr-memory-limit-mb {corr_mem_lim} --stereo-algorithm {stereo_algorithm}"
asp_opt = asp_opt1 + asp_opt2 + asp_opt3


asp_sgm_opt = f' -t rpc --alignment-method local_epipolar ' + asp_opt
asp_args = f' {left_img} {left_xml} {right_img} {right_xml} {output_directory}'

parallel_stereo_call = "/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo"
parallel_stereo_cmd = f'time {parallel_stereo_call} {asp_sgm_opt} {asp_args}'

output_pointcloud = f"{output_directory}*-PC.tif"

p2dem_call = "/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/point2dem"
p2dem_opts = "--tr 4.0 --nodata-value -9999.0 --t_srs EPSG:32616 --errorimage"
p2dem_cmd = f'time {p2dem_call} {p2dem_opts} {output_pointcloud}'

with open(job_fn,'w') as f:
    slurm_keyword = f'#! /bin/bash \n #SBATCH --job-name={os.path.basename(job_fn)} -t 20:00:00 -c10 --mem=190G'
    directory_change = f'cd {parent_dir}'
    out_str = slurm_keyword+"\n"+directory_change+"\n"+parallel_stereo_cmd+"\n"+p2dem_cmd
    f.write(out_str)

## ASP Block

In [21]:
job_fn = f"asp_block_job.sh"
output_directory = os.path.join(stereo_dir,'cpu/asp_block/asp_block_processing')
corr_kernel = 21


## ASP algorithm specific parameters
stereo_algorithm = "asp_bm"
subpix_kernel = 15
subpix_mode = 1
filter_mode = 1 
erode_max_size = 0
cost_mode = 2
corr_mem_lim = 5000

asp_opt1 = f"--corr-kernel {corr_kernel} {corr_kernel} --subpixel-mode {subpix_mode}"
asp_opt2 = f" --subpixel-kernel 15 15 --filter-mode {filter_mode} --cost-mode {cost_mode}"
asp_opt3= f" --corr-memory-limit-mb {corr_mem_lim} --stereo-algorithm {stereo_algorithm}"
asp_opt = asp_opt1 + asp_opt2 + asp_opt3


asp_sgm_opt = f' -t rpc --alignment-method local_epipolar ' + asp_opt
asp_args = f' {left_img} {left_xml} {right_img} {right_xml} {output_directory}'

parallel_stereo_call = "/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo"
parallel_stereo_cmd = f'time {parallel_stereo_call} {asp_sgm_opt} {asp_args}'

output_pointcloud = f"{output_directory}*-PC.tif"

p2dem_call = "/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/point2dem"
p2dem_opts = "--tr 4.0 --nodata-value -9999.0 --t_srs EPSG:32616 --errorimage"
p2dem_cmd = f'time {p2dem_call} {p2dem_opts} {output_pointcloud}'

with open(job_fn,'w') as f:
    slurm_keyword = f'#! /bin/bash \n #SBATCH --job-name={os.path.basename(job_fn)} -t 20:00:00 -c10 --mem=190G'
    directory_change = f'cd {parent_dir}'
    out_str = slurm_keyword+"\n"+directory_change+"\n"+parallel_stereo_cmd+"\n"+p2dem_cmd
    f.write(out_str)